In [1]:
import pandas as pd
import json
import ast
import configparser
import pymysql

In [2]:
def pipeline(path):
    """
    Function performs required steps to:
    1) clean dataframe
    2) add ingestion timestamp column
    3) prepare dataframe data to insert to MySQL database
    """
    df = pd.read_csv(path)
    df.fillna('', inplace=True)
    df["ingestion_ts"] = pd.to_datetime(pd.Timestamp.today())
    df = df.values.tolist()
    
    return df

In [3]:
path = 'data/ready_for_ingestion/movies_metadata.csv'
movies_metadata_to_insert = pipeline(path)

C:\Users\filip\AppData\Local\Temp\ipykernel_19564\335438241.py:8: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [7]:
parser = configparser.ConfigParser()
parser.read("config\mysql.conf")
hostname = parser.get("mysql_config", "hostname")
username = parser.get("mysql_config", "username")
password = parser.get("mysql_config", "password")
dbname = parser.get("mysql_config", "database")
port = parser.get("mysql_config", "port")

conn = pymysql.connect(
    host=hostname,
    user=username,
    password=password,
    db=dbname,
    port=int(port)
)

if conn is None:
    print("Error. Connection to MySQL cannot be established.")
else:
    print("Successfully connected to MySQL.")

cur = conn.cursor()

Successfully connected to MySQL.


In [8]:
create_table = """
CREATE TABLE IF NOT EXISTS movies_raw.movies_metadata (
    adult BINARY, 
    budget INT, 
    homepage VARCHAR(65535), 
    movie_id INT, 
    imdb_id VARCHAR(255), 
    original_language VARCHAR(2),
    original_title VARCHAR(255), 
    overview VARCHAR(65535), 
    popularity FLOAT, 
    poster_path VARCHAR(65535),
    release_date DATE, 
    revenue INT, 
    runtime INT, 
    status VARCHAR(50), 
    tagline VARCHAR(65535), 
    title VARCHAR(255),
    video BINARY, 
    vote_average FLOAT, 
    vote_count INT,
    ingestion_ts TIMESTAMP
);
"""

insert_query = """
INSERT INTO movies_raw.movies_metadata (adult, budget, homepage, movie_id, imdb_id, original_language,
       original_title, overview, popularity, poster_path,
       release_date, revenue, runtime, status, tagline, title,
       video, vote_average, vote_count, ingestion_ts)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

In [9]:
cur.execute(create_table)

0

In [10]:
# Wstawienie danych do tabeli
cur.executemany(insert_query, movies_metadata_to_insert)

# Zatwierdzenie zmian w bazie danych
conn.commit()

# Zamknięcie kursora i połączenia
cur.close()
conn.close()